#Variational Quantum Eigensolver & QAOA / Classical setting part

Now we see a setting of classical optimization part and some speed up on backend.

##Classical Optimization
Inside of the variational method, the steps for the algorithm is,

1. Short parametrized quantum circuit
2. Calculate expectation value using the result of the circuit. And then give the next proposed parameter using classical optimization algorithm.

In [1]:
!pip install blueqat

     |████████████████████████████████| 51kB 8.1MB/s 


In [2]:
import numpy as np
from blueqat import Circuit, vqe
from blueqat.pauli import X, Y, Z, I
from blueqat.pauli import qubo_bit as q
from blueqat.vqe import AnsatzBase, Vqe

class QubitAnsatz(AnsatzBase):
    def __init__(self, hamiltonian):
        super().__init__(hamiltonian, 4)
        self.step = 1

    def get_circuit(self, params):
        a, b, c, d = params
        return Circuit().ry(a)[0].rz(b)[0].ry(c)[1].rz(d)[1]

h = -3*q(0)-3*q(1)-2*q(0)*q(1)
h = h.to_expr().simplify()
minimizer=vqe.get_scipy_minimizer(method="COBYLA",options={"tol":5.0e-4})
runner = Vqe(QubitAnsatz(h),minimizer=minimizer)
result = runner.run()

print('Result by VQE')
print(runner.ansatz.get_energy(result.circuit, runner.sampler))

# Hamiltonian to matrix
mat = h.to_matrix()

# Calculate by numpy
print('Result by numpy')
print(np.linalg.eigh(mat)[0][0])

Result by VQE
-7.999999845198036
Result by numpy
-8.0



Scipy is used for the default optimization and possible to change it from the list.

https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html

'Nelder-Mead', 'Powell', 'CG', 'BFGS', 'L-BFGS-B', 'TNC', 'COBYLA', 'SLSQP'

And if you want to use your own optimizer you can use it by making a function.

```python
def hyperopt_minimizer(objective, n_params):
    from hyperopt import fmin, Trials, tpe, hp
    trials = Trials()
    best = fmin(objective, [hp.uniform(f'p{i}', 0., 2 * np.pi) for i in range(n_params)],
            algo=tpe.suggest, max_evals=10000, trials=trials, verbose=1)
    return list(best.values())
```

##Numba backend
Blueqat has a numba backend to achieve simple speed up on long quantum circuit.

In [0]:
from blueqat import BlueqatGlobalSetting
BlueqatGlobalSetting.set_default_backend('numpy')

In [0]:
import time
from blueqat import vqe
from blueqat.pauli import qubo_bit as q

hamiltonian = q(0)*q(1)*q(2)*q(3)
step = 10

start = time.time()
result = vqe.Vqe(vqe.QaoaAnsatz(hamiltonian, step)).run()
end = time.time()

print(end - start)

20.231501579284668


This is 10steps VQE. Now we see numba backend on the same hamiltonian.

In [0]:
from blueqat import BlueqatGlobalSetting
BlueqatGlobalSetting.set_default_backend('numba')

In [0]:
import time
from blueqat import vqe
from blueqat.pauli import qubo_bit as q

hamiltonian = q(0)*q(1)*q(2)*q(3)
step = 10

start = time.time()
result = vqe.Vqe(vqe.QaoaAnsatz(hamiltonian, step)).run()
end = time.time()

print(end - start)